In [1]:
import pandas as pd
import io
import json

In [2]:
import boto3


# Set your desired bucket name and AWS region
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
bucket_name = 'wildscan-camera-data'

print(bucket_name)


wildscan-camera-data


In [3]:
s3_client = boto3.client('s3')
object_key = ('annotations/cct_labels.json')
try:
    metadata_object = s3_client.get_object(Bucket=bucket_name, Key=object_key)
    print("Access to object confirmed.")
except ClientError as e:
    print(f"Error: {e}")

Access to object confirmed.


In [19]:
img_rel_path = '1/5998cfa4-23d2-11e8-a6a3-ec086b02610b_0.jpg'
filename = img_rel_path.split('/', 1)[1]
filename

'5998cfa4-23d2-11e8-a6a3-ec086b02610b_0.jpg'

In [35]:
def s3_split_from_json(source_bucket, json_key, source_prefix, target_bucket, target_prefix):
    s3 = boto3.client('s3')
    
    # 1. Download JSON split configuration
    obj = s3.get_object(Bucket='sagemaker-us-east-1-868702911043', Key='group4_project_trials/data_split.json')
    split_info = json.loads(obj['Body'].read().decode('utf-8'))
    #split_info = local_dict
    
    # 2. Process each image record
    for img_rel_path, split_type in split_info.items():
        img_filename = img_rel_path.split('/', 1)[1]
        src_key = f"{source_prefix}/{img_filename}".strip('/')
        dest_key = f"{target_prefix}/{split_type}/{img_rel_path}".strip('/')

        #print(f"src_key: {src_key}   dest_key: {dest_key}")
        # 3. Copy between S3 paths directly
        s3.copy_object(
            CopySource={'Bucket': source_bucket, 'Key': src_key},
            Bucket=target_bucket,
            Key=dest_key
        )
        # Get the object from the source bucket (streaming mode)
        #print(f"src_key: {src_key}")
        #response = s3.get_object(Bucket=source_bucket, Key=src_key)

        # Stream data directly to destination bucket
        #s3.put_object(
        #    Bucket=target_bucket,
        #    Key=dest_key,
        #    Body=response['Body'],  # Streams data directly
        #    Metadata=response.get('Metadata', {}),
        #    ContentType=response['ContentType']
        #)
        
        #print(f"Copied {src_key} -> {dest_key}")

In [36]:
# source bucket and prefix
source_bucket = bucket_name
source_prefix = 'cct_resized'

# target bucket and prefix
target_bucket = 'sagemaker-us-east-1-868702911043'
#target_bucket = source_bucket
target_prefix = f'{source_prefix}_data_split' # no write access under cct_resized

In [37]:
# json_key = (f'{target_prefix}/data_split.json') update later, not used for now, downloaded directly from geoffrey s3 bucket
s3_split_from_json(source_bucket, json_key, source_prefix, target_bucket , target_prefix)

KeyboardInterrupt: 